In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, Conv2DTranspose
from keras.constraints import max_norm
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from pathlib import Path
from glob import glob
import os

In [6]:
# tento di scrivere una funzione che ricostruisca l'immagine.
# step 1: multiplo esatto delle tile size

# TODO: restructure code

# params ===
#tile_size=256
#model = keras.models.load_model("model_dset2_56px_neuralnet_vanilla")


def predict_tiles_from_image(image, model, tile_size=56):
    """ This gives back the denoised <tiles>, according to the loaded <model>
    The model operates on multiple tiles at once. All tiles are shaped into a form
    that the model was trained for, then all put into a np.array container.
    This is the way the models expects the tiles for the prediction.
    
    NOTE: This function relies on crop_in_tiles() function.
    
    params
    ======
    
    <image>: a pillow Image object
    <model>: a keras trained model
    """
    to_predict = [
        x.reshape(tile_size, tile_size, 1) for x in crop_in_tiles(image, tile_size)
    ]
    
    to_predict = np.array(to_predict)
    
    return model.predict(to_predict)
    

# from Dataset class
def crop_in_tiles(image, tile_size=56, shift=0, asarray=True):

    """
    This generator function crops an image in several tiles
    tile_size × tile_size squares, yielding a tile
    every iteration.

    If the input image is not a perfect multiple of
    a(tile_size) × b(tile_size), non-square tiles are NOT
    YIELDED.

    params
    ======

    image: a Pillow open image
    tile_size: <int> pixels; size of the tile side
    shift: <int>: the offset from 0,0 in pixels
    """

    assert isinstance(tile_size, int)
    assert isinstance(shift, int)

    width, height = image.size

    #calculate coordinates of every tile
    for x in range (0+shift, width, tile_size):
        if width - x < tile_size:
            continue

        for y in range (0+shift, height, tile_size):
            if height - y < tile_size:
                continue

            # tile coord ===
            tile_coord = (
                x, y, # upper left coords
                x + tile_size, y + tile_size # lower right coords
            )

            tile = image.crop(tile_coord)
            
            if not asarray:
                yield tile #yielding tile as image
            else:
                yield np.array(tile).astype("float32") / 255

                
def image_rebuilder(image, model, tile_size=56, verbose=True):
    """ Takes an input image, splits into three channels.
    Each channel is broken down into <tile_size> chunks, predicted and reassembled.
    """

    def say(*args, **kwargs):
        if verbose:
            print(*args, **kwargs)
    
    # TODO: here splitting into channels
    channels = [image] # will contain three Images, one for channel
    
    width, height = channels[0].size #all three channels have the same size
    say(f"width: {width}; height: {height}")
    
    # TODO
    # for now, we support only exact multiples of tile_size
    tile_width = int(width / tile_size)
    tile_height = int(height / tile_size)

    say(f"Image multiple of {tile_width}×{tile_height} integer tiles.")
    
    for i, channel in enumerate(channels):
        say(f"Processing channel {i + 1} of {len(channels)}")
        
        # TODO: tutte assieme
        pred_tiles = [predict_tiles_from_image(channel, model, tile_size=tile_size)]
                
        say(f"Predicted tiles length: {len(pred_tiles[0])}")
        
        # now we need to rebuild a numpy array based on the tile_width*tile_height original geometry        
        gen = (x for x in pred_tiles[0])
        
        # the final assembly is very fast ===
        returnimage = []

        #for i in range(tile_height):
        #    row_tiles = next(gen)
        #    for j in range(tile_width - 1):
        #        next_tile = next(gen)
        #        row_tiles = np.concatenate((row_tiles, next_tile), axis=1)
        #    returnimage.append(row_tiles)
        #
        #returnimage = np.array(returnimage)
        #returnimage = np.vstack(returnimage)
        
        for i in range(tile_width):
            row_tiles = next(gen)
            for j in range(tile_height - 1):
                next_tile = next(gen)
                row_tiles = np.concatenate((row_tiles, next_tile), axis=0)
            returnimage.append(row_tiles)
        
        returnimage = np.array(returnimage)
        returnimage = np.hstack(returnimage)
        
        # from array to Image
        returnimage = np.uint8(returnimage * 256)
        
        # TODO: the three channels
        return Image.fromarray(returnimage[:,:,0])        
        

In [7]:
!ls

0.jpg
1.jpg
15_epochs
2.jpg
3.jpg
4.jpg
5.jpg
Building an Image Denoiser with a Keras autoencoder neural network.pdf
Denoiser.ipynb
Denoiser_with_own_dataset-LAPTOP-BD7NICPD.ipynb
Denoiser_with_own_dataset.ipynb
Rebuilder.ipynb
dataset shooting info.xlsx
lino
model_128px_neuralnet_128_64_64_128_kernel3x3
model_1_chan_B_ts_512.png
model_1_chan_G_ts_512.png
model_1_chan_R_ts_512.png
model_2_chan_R_ts_512.png
model_56px_neuralnet_128_64_64_128_kernel3x3
model_56px_neuralnet_128_64_64_128_kernel6x6
model_56px_neuralnet_vanilla
model_ds2_56px_neuralnet_vanilla
model_dset2_56px_neuralnet_vanilla
standard_dataset
test 168×112px.png
test_tiles


In [8]:
# TODO: put all this within functions, classes even better
model1 = keras.models.load_model("model_56px_neuralnet_128_64_64_128_kernel3x3")
model2 = keras.models.load_model("model_128px_neuralnet_128_64_64_128_kernel3x3")
model3 = keras.models.load_model("model_dset2_56px_neuralnet_vanilla")


img = Image.open("standard_dataset/dataset_0004_ISO1600.JPG")

In [9]:
ts = 512 # +128
print(f"Tile size: {ts}")

for i, model in enumerate([model1, model2, model3]):
    for channel in "RGB":
        rebuilt = image_rebuilder(img.getchannel(channel), model, tile_size=ts)
        
        outname = f"model_{i+1}_chan_{channel}_ts_{str(ts)}.png"
        rebuilt.save(outname)

Tile size: 512
width: 5184; height: 3456
Image multiple of 10×6 integer tiles.
Processing channel 1 of 1
Predicted tiles length: 60
width: 5184; height: 3456
Image multiple of 10×6 integer tiles.
Processing channel 1 of 1
Predicted tiles length: 60
width: 5184; height: 3456
Image multiple of 10×6 integer tiles.
Processing channel 1 of 1
Predicted tiles length: 60
width: 5184; height: 3456
Image multiple of 10×6 integer tiles.
Processing channel 1 of 1
Predicted tiles length: 60
width: 5184; height: 3456
Image multiple of 10×6 integer tiles.
Processing channel 1 of 1
Predicted tiles length: 60
width: 5184; height: 3456
Image multiple of 10×6 integer tiles.
Processing channel 1 of 1
Predicted tiles length: 60
width: 5184; height: 3456
Image multiple of 10×6 integer tiles.
Processing channel 1 of 1
Predicted tiles length: 60
width: 5184; height: 3456
Image multiple of 10×6 integer tiles.
Processing channel 1 of 1
Predicted tiles length: 60
width: 5184; height: 3456
Image multiple of 10×6 

In [13]:
png = sorted(glob("*.png"))
png

['model_1_chan_B_ts_512.png',
 'model_1_chan_G_ts_512.png',
 'model_1_chan_R_ts_512.png',
 'model_2_chan_B_ts_512.png',
 'model_2_chan_G_ts_512.png',
 'model_2_chan_R_ts_512.png',
 'model_3_chan_B_ts_512.png',
 'model_3_chan_G_ts_512.png',
 'model_3_chan_R_ts_512.png',
 'test 168×112px.png']

In [11]:
for i in range(3):
    r = Image.open(f"model_{i+1}_chan_R_ts_512.png")
    g = Image.open(f"model_{i+1}_chan_G_ts_512.png")
    b = Image.open(f"model_{i+1}_chan_B_ts_512.png")
    r
        

['model_1_chan_B_ts_512.png',
 'model_1_chan_G_ts_512.png',
 'model_1_chan_R_ts_512.png',
 'model_2_chan_B_ts_512.png',
 'model_2_chan_G_ts_512.png',
 'model_2_chan_R_ts_512.png',
 'model_3_chan_B_ts_512.png',
 'model_3_chan_G_ts_512.png',
 'model_3_chan_R_ts_512.png',
 'test 168×112px.png']